# 🇻🇳 GPT-SoVITS Vietnamese - Colab
Phiên bản tối ưu hóa cho tiếng Việt. Chạy các cell bên dưới theo thứ tự.

**Lưu ý:** Trước khi chạy, vào menu **Runtime** > **Change runtime type** > chọn **T4 GPU**.

## 1️⃣ Cài đặt môi trường & Tải Model
Chạy cell này và đợi khoảng 5-7 phút.

In [ ]:
# @title Cài đặt & Tải Model

# 1. Install System Dependencies
!apt-get update
!apt-get install -y ffmpeg

# 2. Clone Repository (Nếu chưa có)
import os
if not os.path.exists("GPT-SoVITS-Vietnamese"):
    !git clone https://github.com/TuanNam/GPT-SoVITS-Vietnamese.git
    %cd GPT-SoVITS-Vietnamese
else:
    %cd GPT-SoVITS-Vietnamese
    !git pull

# 3. Install Python Dependencies
!pip install -r requirements.txt
!pip install faster-whisper funasr modelscope ffmpeg-python huggingface_hub

# 4. Download Pretrained Models
!python download_models_vi.py

print("✅ Cài đặt hoàn tất!")

## 2️⃣ Chạy WebUI (Giao diện Tiếng Việt)
Sau khi chạy, click vào link public URL (ví dụ: `https://xxxx.gradio.live`).

In [ ]:
# @title Khởi động WebUI
!python webui.py

## 3️⃣ (Tùy chọn) Sao lưu Model vào Google Drive
Kết nối Google Drive để lưu lại các model đã train.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy SoVITS weights
!mkdir -p /content/drive/MyDrive/GPT-SoVITS-Backup/SoVITS_weights
!cp -r SoVITS_weights_v2/* /content/drive/MyDrive/GPT-SoVITS-Backup/SoVITS_weights/

# Copy GPT weights
!mkdir -p /content/drive/MyDrive/GPT-SoVITS-Backup/GPT_weights
!cp -r GPT_weights_v2/* /content/drive/MyDrive/GPT-SoVITS-Backup/GPT_weights/

print("✅ Đã sao lưu thành công!")